In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp
from qiskit.synthesis import LieTrotter
from qiskit.primitives import StatevectorEstimator
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_ibm_runtime.fake_provider import FakeTorino
import numpy as np

# using 5 qubits locally (20 is too heavy for statevector)
n = 5

# parameters
Jx, Jy, Jz = 0.5, 0.3, 0.7
hx, hy, hz = 0.2, 0.1, 0.4
t = 0.2
steps = 2

# build Hamiltonian
pstr = []
coeff = []

# nearest-neighbour terms
for i in range(n-1):
    L = 'I'*(n-i-2)
    R = 'I'*i
    pstr += [L+"XX"+R, L+"YY"+R, L+"ZZ"+R]
    coeff += [Jx, Jy, Jz]

# single-qubit fields
for i in range(n):
    L = 'I'*(n-i-1)
    R = 'I'*i
    pstr += [L+"X"+R, L+"Y"+R, L+"Z"+R]
    coeff += [hx, hy, hz]

H = SparsePauliOp(pstr, coeff)
print("terms in H:", len(pstr))

# time evolution circuit
qc = QuantumCircuit(n)
evo = PauliEvolutionGate(H, time=t, synthesis=LieTrotter(reps=steps))
qc.append(evo, range(n))

print("circuit:", qc.num_qubits, "| depth =", qc.depth())

# observable sum(Z)
obs_terms = []
for i in range(n):
    obs_terms.append('I'*(n-i-1) + 'Z' + 'I'*i)

obs = SparsePauliOp(obs_terms, [1]*n)
print("\nobservable = sum Z_i")

# ideal expectation
est = StatevectorEstimator()
out = est.run([(qc, obs)]).result()
ideal = float(out[0].data.evs)
print("ideal =", f"{ideal:.6f}")

# layouts (scaled-down versions of original)
backend = FakeTorino()

layouts = {
    "L1": list(range(n)),
    "L2": list(range(10, 10+n)),
    "L3": list(range(20, 20+n))
}

tcircs = {}
print("using backend:", backend.name)

for name, lay in layouts.items():
    print("\n", name, lay)
    tc = transpile(
        qc,
        backend=backend,
        initial_layout=lay,
        optimization_level=1,
        seed_transpiler=42
    )
    tcircs[name] = tc
    print("depth:", tc.depth(), "| gates:", tc.size())

# noisy simulation (not using transpiled circuits because they exceed memory)
results = {}

for name in layouts:
    print("\n", name)

    nm = NoiseModel()
    nm.add_all_qubit_quantum_error(
        depolarizing_error(0.001, 1),
        ['u','u1','u2','u3','rz','sx','x','id']
    )
    nm.add_all_qubit_quantum_error(
        depolarizing_error(0.01, 2),
        ['cx','cy','cz','ecr']
    )

    sim = AerSimulator(noise_model=nm)

    # use original circuit; the transpiled ones blow up in size
    dqc = qc.decompose()
    mqc = dqc.copy()
    mqc.measure_all()

    try:
        job = sim.run(mqc, shots=1000, seed_simulator=42)
        res = job.result()
        counts = res.get_counts()

        tot = sum(counts.values())
        exp = 0

        for b, c in counts.items():
            ones = b.count('1')
            parity = n - 2*ones
            exp += parity * c / tot

        results[name] = exp
        print("noisy =", f"{exp:.6f}")
        print("err   =", f"{abs(exp - ideal):.6f}")

    except Exception as e:
        print("failed:", e)
        results[name] = None

# comparison
print("\nresults:")
errs = {}
for name, val in results.items():
    if val is not None:
        e = abs(val - ideal)
        errs[name] = e
        print(name, ":", f"{val:.6f}", "err =", f"{e:.6f}")

if errs:
    best = min(errs, key=errs.get)
    print("\nbest layout:", best)

# extra info
print("\nstats:")
print("original depth:", qc.depth())

if errs:
    bc = tcircs[best]
    print("best layout depth:", bc.depth())
    print("gate count:", bc.size())
    print("2q gates:", bc.num_nonlocal_gates())


terms in H: 27
circuit: 5 | depth = 1

observable = sum Z_i
ideal = 4.954876
using backend: fake_torino

 L1 [0, 1, 2, 3, 4]
depth: 157 | gates: 269

 L2 [10, 11, 12, 13, 14]
depth: 157 | gates: 269

 L3 [20, 21, 22, 23, 24]
depth: 157 | gates: 269

 L1
noisy = 4.954000
err   = 0.000876

 L2
noisy = 4.954000
err   = 0.000876

 L3
noisy = 4.954000
err   = 0.000876

results:
L1 : 4.954000 err = 0.000876
L2 : 4.954000 err = 0.000876
L3 : 4.954000 err = 0.000876

best layout: L1

stats:
original depth: 1
best layout depth: 157
gate count: 269
2q gates: 48
